In [91]:
pip install nba_api

Note: you may need to restart the kernel to use updated packages.


In [100]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [102]:
# 步行者队伍ID（NBA官方编号）
PACERS_TEAM_ID = '1610612754'

# 抓取2024-25赛季步行者所有比赛（含季后赛）
gamefinder = leaguegamefinder.LeagueGameFinder(
    team_id_nullable=PACERS_TEAM_ID,
    season_nullable='2024-25'
)

# 提取数据框
games_df = gamefinder.get_data_frames()[0]

# 显示前几行
print(games_df[['GAME_DATE', 'MATCHUP', 'WL', 'PTS', 'PLUS_MINUS']].head())

# 可选：保存到CSV（你也可以只用作中间缓存，不上传）
games_df.to_csv("pacers_2024_25_games.csv", index=False)


    GAME_DATE      MATCHUP WL  PTS  PLUS_MINUS
0  2025-06-13  IND vs. OKC  L  104        -7.0
1  2025-06-11  IND vs. OKC  W  116         9.0
2  2025-06-08    IND @ OKC  L  107       -16.0
3  2025-06-05    IND @ OKC  W  111         1.0
4  2025-05-31  IND vs. NYK  W  125        17.0


In [104]:
# ✅ Step 1: 日期排序（模型需要时间顺序）
games_df['GAME_DATE'] = pd.to_datetime(games_df['GAME_DATE'])
games_df.sort_values('GAME_DATE', inplace=True)
games_df.reset_index(drop=True, inplace=True)

# ✅ Step 2: 特征工程（构造新变量）
games_df['IS_HOME'] = games_df['MATCHUP'].apply(lambda x: 1 if 'vs.' in x else 0)
games_df['WIN'] = games_df['WL'].apply(lambda x: 1 if x == 'W' else 0)
games_df['POINT_DIFF'] = games_df['PTS'] - games_df['PTS'].shift(1)

# ✅ Step 3: Rolling 特征（用最近5场的平均来预测）
games_df['ROLLING_PTS'] = games_df['PTS'].rolling(window=5, min_periods=1).mean()
games_df['ROLLING_PLUS_MINUS'] = games_df['PLUS_MINUS'].rolling(window=5, min_periods=1).mean()
games_df['ROLLING_WIN_RATE'] = games_df['WIN'].rolling(window=5, min_periods=1).mean()

# ✅ Step 4: 是否是季后赛
games_df['IS_PLAYOFF'] = games_df['GAME_ID'].astype(str).apply(lambda x: 1 if x.startswith('424') else 0)


# ✅ Step 5: 筛选最终用于建模的变量
model_df = games_df[[
    'GAME_DATE', 'MATCHUP', 'IS_HOME', 'WIN', 'POINT_DIFF',
    'ROLLING_PTS', 'ROLLING_PLUS_MINUS', 'ROLLING_WIN_RATE', 'IS_PLAYOFF'
]]

# ✅ Step 6: 可选保存
model_df.to_csv("pacers_model_ready.csv", index=False)

# ✅ Step 7: 查看部分结果
print(model_df.tail())


     GAME_DATE      MATCHUP  IS_HOME  WIN  POINT_DIFF  ROLLING_PTS  \
101 2025-05-31  IND vs. NYK        1    1        31.0        112.6   
102 2025-06-05    IND @ OKC        0    1       -14.0        112.0   
103 2025-06-08    IND @ OKC        0    0        -4.0        113.4   
104 2025-06-11  IND vs. OKC        1    1         9.0        110.6   
105 2025-06-13  IND vs. OKC        1    0       -12.0        112.6   

     ROLLING_PLUS_MINUS  ROLLING_WIN_RATE  IS_PLAYOFF  
101                 1.6               0.6           0  
102                 0.8               0.6           0  
103                -1.2               0.6           0  
104                -1.2               0.6           0  
105                 0.8               0.6           0  


In [106]:
# 1. Load dataset
df = pd.read_csv("pacers_2024_25_games.csv")

# 2. Create IS_PLAYOFF flag
df['IS_PLAYOFF'] = df['GAME_ID'].astype(str).apply(lambda x: 1 if x.startswith('424') else 0)

# 3. Feature engineering
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
df = df.sort_values('GAME_DATE').reset_index(drop=True)
df['IS_HOME'] = df['MATCHUP'].apply(lambda x: 1 if 'vs.' in x else 0)
df['WIN'] = df['WL'].apply(lambda x: 1 if x == 'W' else 0)
df['POINT_DIFF'] = df['PTS'] - df['PTS'].shift(1)
df['ROLLING_PTS'] = df['PTS'].rolling(window=5, min_periods=1).mean()
df['ROLLING_PLUS_MINUS'] = df['PLUS_MINUS'].rolling(window=5, min_periods=1).mean()
df['ROLLING_WIN_RATE'] = df['WIN'].rolling(window=5, min_periods=1).mean()

# 4. Advanced feature 1: Opponent winrate
df['OPP_TEAM'] = df['MATCHUP'].str.extract(r'@ (.*)|vs\. (.*)')[0].fillna(df['MATCHUP'].str.extract(r'@ (.*)|vs\. (.*)')[1])
opp_strength = df.groupby('OPP_TEAM')['WIN'].mean().to_dict()
df['OPP_AVG_WINRATE'] = df['OPP_TEAM'].map(opp_strength)

# 5. Advanced feature 2: Game pace estimation
df['PACE_ESTIMATE'] = df['PTS'] + df['PLUS_MINUS'].apply(lambda x: -x)

# 6. Select features
features = [
    'IS_HOME', 'POINT_DIFF', 'ROLLING_PTS',
    'ROLLING_PLUS_MINUS', 'ROLLING_WIN_RATE',
    'OPP_AVG_WINRATE', 'PACE_ESTIMATE'
]
X = df[features].fillna(0)
y = df['WIN']

# 7. Train/test split
X_train = X[df['IS_PLAYOFF'] == 0]
X_test = X[df['IS_PLAYOFF'] == 1]
y_train = y[df['IS_PLAYOFF'] == 0]
y_test = y[df['IS_PLAYOFF'] == 1]

# 8. Fit model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# 9. Evaluate
accuracy = accuracy_score(y_test, y_pred)
print("🎯 Accuracy on Playoff Games:", accuracy)
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred))


🎯 Accuracy on Playoff Games: 0.8

📊 Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.50      0.60         6
           1       0.81      0.93      0.87        14

    accuracy                           0.80        20
   macro avg       0.78      0.71      0.73        20
weighted avg       0.79      0.80      0.79        20



In [108]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 初始化模型
lr_model = LogisticRegression(max_iter=1000)

# 拟合训练集（常规赛）
lr_model.fit(X_train, y_train)

# 预测季后赛
lr_pred = lr_model.predict(X_test)

# 输出结果
lr_accuracy = accuracy_score(y_test, lr_pred)
print("📈 Logistic Regression Accuracy on Playoff Games:", lr_accuracy)
print("\n📊 Classification Report:")
print(classification_report(y_test, lr_pred))

📈 Logistic Regression Accuracy on Playoff Games: 0.8

📊 Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.83      0.71         6
           1       0.92      0.79      0.85        14

    accuracy                           0.80        20
   macro avg       0.77      0.81      0.78        20
weighted avg       0.83      0.80      0.81        20

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'deprecated', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [89]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Load processed dataset
df = pd.read_csv("pacers_model_ready.csv")

# Define features and target
features = ['IS_HOME', 'POINT_DIFF', 'ROLLING_PTS', 'ROLLING_PLUS_MINUS', 'ROLLING_WIN_RATE']
X = df[features].fillna(0)
y = df['WIN']

# Train/test split: train on regular season (IS_PLAYOFF == 0)
X_train = X[df['IS_PLAYOFF'] == 0]
y_train = y[df['IS_PLAYOFF'] == 0]

# Initialize models
rf_model = RandomForestClassifier(random_state=42)
lr_model = LogisticRegression(max_iter=1000)

# Fit models
rf_model.fit(X_train, y_train)
lr_model.fit(X_train, y_train)

# Use most recent 5 games to construct G5 and G6 features
recent_games = df.tail(5)

rolling_pts = recent_games['ROLLING_PTS'].mean()
rolling_pm = recent_games['ROLLING_PLUS_MINUS'].mean()
rolling_wr = recent_games['ROLLING_WIN_RATE'].mean()
point_diff = recent_games.iloc[-1]['POINT_DIFF']

# G5: Away game
g5_input = pd.DataFrame([{
    'IS_HOME': 0,
    'POINT_DIFF': point_diff,
    'ROLLING_PTS': rolling_pts,
    'ROLLING_PLUS_MINUS': rolling_pm,
    'ROLLING_WIN_RATE': rolling_wr
}])

# Fill NaNs
g5_input = g5_input.fillna(0)

# Predict
g5_rf = rf_model.predict(g5_input)[0]
g5_lr = lr_model.predict(g5_input)[0]

# Output
results = {
    "G5 Random Forest": "Win" if g5_rf == 1 else "Loss",
    "G5 Logistic Regression": "Win" if g5_lr == 1 else "Loss",
}

print(results)


{'G5 Random Forest': 'Loss', 'G5 Logistic Regression': 'Loss'}
